Consider an infinitely repeated duopoly market with stochastic demand. Specifically, firms 1 and 2 interact over time $t \in\{1, \ldots, \infty\}$ with the demand in each time period being either high (H) or low (L). If the demand is $\mathrm{H}$, then the inverse demand function is given by $p_{H}=120-q_{1}-q_{2} ;$ if the demand is $L,$ then the inverse demand function is given by $p_{L}=60-q_{1}-q_{2}$ (where $q_{1}$ is quantity chosen by firm $1,$ and $q_{2}$ is the quantity chosen by firm 2). Furthermore, suppose that firms discount the future according to the discount rate of $\beta=.99$ and the demand evolves according to the transition probability matrix $M=\left[\begin{array}{rr}.8 & .2 \\ .3 & .7\end{array}\right] .$ For example, the probability that the demand in period $t+1$ is $\mathrm{H}$ given that the demand in period $t$ is $\mathrm{H}$ is . $8 .$ For simplicity, suppose that each firm produces at zero marginal cost.

a) Suppose that firm 1 's policy is to produce 30 in every period. Determine the value function and the optimal policy functions for firm $2 .$ Hints: there are two possible states to consider: $(\mathrm{H})$ and $(\mathrm{L}) .$ Your goal is to find the value of being in each state and the optimal decision by firm 2 .

b) Suppose that firm 1 's policy is to produce 25 in every round as long neither firm exceeded 35 in any round in the past. If either of the firms exceeded $35,$ firm 1 will produce $40 .$ Determine the value and the policy function for firm $2 .$ Hint: there are four possible states to consider: (H, neither firm exceeded 35), (H, one of the firms exceeded
35), (L, neither firm exceeded exceeded 35), (L, one of the firms exceeded 35)

In [1]:
import numpy as np

M = np.array([[0.8, 0.2], [0.3, 0.7]])

p_h = lambda q1, q2: 120 - q1 - q2
p_l = lambda q1, q2: 60 - q1 - q2

beta = 0.99

H = 0
L = 1

## a

For firm 1, the policy is 
$
\pi_1(a|s) = 30
$.
Given any state $s$, the output action $a$ of $\pi_1$ is 30. 


For firm 2, the state space is $S = \{H, L\}$  
The action space is $A = \{a | a \in \mathbb{R} \land 0 \leq a < 90\}$  
We want to get the policy of firm 2, $\pi_2$, with value iteration.  

Value function for $\pi_2$ is 

$$
\begin{align}
V(s) &= \underset{a}{max} \sum_{s' \in S} \mathcal{T} (s, a, s') (R(s, a, s') + \beta V(s'))\\
     % &= \underset{a}{max} \sum_{s' \in S} M (R(s, a, s') + \beta V(s'))
\end{align}
$$

Given current state $s$ and the action going to take $a$, $\mathcal{T}$ returns the probability of which the ongoing state is $s'$. $\mathcal{T}$ can be gotten by querying $M$.   
Similarly, $R$ returns the reward when taking action $a$ in state $s$ and arriving state $s'$.   

Once we get the converged (optimal) value function $V^*$, we have $$\pi_2(a|s) = \underset{a}{argmax} \sum_{s' \in S} \mathcal{T} (s, a, s') (R(s, a, s') + \beta V^*(s'))$$ 

In this problem, the action will not affect the state transform, so we can simplify the $\mathcal{T}(s, a, s')$ to $\mathcal{T}(s, s')$

In [2]:
def trans_a(s0, s1):
    return M[s0][s1]

The reward function does not care about current state, so we can simplify the $R(s, a, s')$ to $R(a, s')$

In [3]:
# only consider revenue
def rew_a(a, s1):
    assert s1 in [0, 1]

    if s1 == 0:  # next state is H
        return p_h(30, a) * a
    else:  # next state is L
        return p_l(30, a) * a

Value function and $\pi_2$

In [4]:
v_a = np.zeros(2)  # [0]: H, [1]: L

def bf_argmax_a(fn, l, h):
    # include upper boundary h
    ind = np.arange(l, h + 1)
    vs = fn(ind)
    return l + np.argmax(vs), np.max(vs)

def pi_2_a(s):
    # TODO: v_a can be simplified as matrix product
    v_f = lambda a: trans_a(s, 0) * (rew_a(a, 0) + beta * v_a[0]) + trans_a(s, 1) * (rew_a(a, 1) + beta * v_a[1])
    return bf_argmax_a(v_f, 1, 90)

In [5]:
def iteration_a(max_iter, delta):
    for _ in range(max_iter):
        new_v_0 = pi_2_a(0)[1]
        new_v_1 = pi_2_a(1)[1]
        new_v = np.array([new_v_0, new_v_1])
        
        global v_a
        diff = np.max(new_v - v_a)
        if diff <= delta: 
            print("converged")
            break
        
        v_a = new_v

In [6]:
iteration_a(10000, 0.01)

converged


In [7]:
print(f"If current state is H, firm 2 should produce: {pi_2_a(0)[0]}")
print(f"If current state is L, firm 2 should produce: {pi_2_a(1)[0]}")

If current state is H, firm 2 should produce: 39
If current state is L, firm 2 should produce: 24


# b

The value function of firm 2 did not change here, 
$$
\begin{align}
V(s) &= \underset{a}{max} \sum_{s' \in S} \mathcal{T} (s, a, s') (R(s, a, s') + \beta V(s'))\\
     % &= \underset{a}{max} \sum_{s' \in S} M (R(s, a, s') + \beta V(s'))
\end{align}
$$                   
But the $\mathcal{T}(s, a, s')$ and the $R(s, a, s')$ changed, $\mathcal{T}(s, a, s')$ and $R(s, a, s')$ depends on all of the $s, a$ and $s'$ now. 

Let's encode the states first  
$s_0$: (H, neither firm exceeded 35)  
$s_1$: (H, one of the firms exceeded 35)  
$s_2$: (L, neither firm exceeded exceeded 35)  
$s_3$: (L, one of the firms exceeded 35)  

It seems that here is still a problem, 

For $\mathcal{T}(s, a, s')$, if $a > 35$, $s'$ can only be $s_1$ and $s_3$, and this can be gotten by querying the $M$. In this case $s'$ cannot be $s_0$ or $s_2$, the probability for that $s$ be these states will be 0.  
If $a \leq 35$, and neither firm exceeded 35. $s'$ can only be $s_0$ and $s_2$, and this can be gotten by querying the $M$. If  $a \leq 35$, and one of the firms exceeded 35. $s'$ can only be $s_1$ and $s_3$, and this can be gotten by querying the $M$. Other transformation is not allowed, thus has probability 0. 

In [8]:
def trans_b(s0, a, s1):
    if a > 35:
        if s1 == 1 or s1 == 3:
            return M[s0//2][s1//2]
        else:
            return 0
    else:
        if s0 == 1 or s0 == 3: 
            if s1 == 1 or s1 == 3:
                return M[s0//2][s1//2]
            else:
                return 0
        else:
            if s1 == 0 or s1 == 2:
                return M[s0//2][s1//2]
            else:
                return 0

For $R(s, a, s')$, the quantity of firm 1 will changed based on the current state $s$. If $s$ is $s_1$ or $s_3$, firm 1 will produce 40, otherwise, it will produce 25.  

In [9]:
def rew_b(s0, a, s1):
    if s0 == 1 or s0 == 3:
        f1_quan = 40
    else: 
        f1_quan = 25
    
    if s1 == 0 or s1 == 1:
        return p_h(f1_quan, a) * a
    else:
        return p_l(f1_quan, a) * a

Value function and $\pi_2$

In [10]:
v_b = np.zeros(4)  

def bf_argmax_b(fn, l, h):
    # include upper boundary h
    ind = np.arange(l, h + 1)
    vs = []
    for i in ind:
        vs.append(fn(i))
    return l + np.argmax(vs), np.max(vs)

def pi_2_b(s0):
    def v_f(a):
        ep_rew = 0.0
        for s1 in range(4):
            ep_rew += trans_b(s0, a, s1) * (rew_b(s0, a, s1) + beta * v_b[s1])
        return ep_rew
    return bf_argmax_b(v_f, 1, 95)

In [11]:
def iteration_b(max_iter, delta):
    for _ in range(max_iter):
        new_v = []
        for s0 in range(4):
            _v = pi_2_b(s0)[1]
            new_v.append(_v)
        new_v = np.array(new_v)
        
        global v_b
        diff = np.max(new_v - v_b)
        if diff <= delta: 
            print("converged")
            break
        
        v_b = new_v

In [12]:
iteration_b(2000, 0.01)

converged


In [13]:
print(f"If current state is s_0 (H, neither firm exceeded 35), firm 2 should produce: {pi_2_b(0)[0]}")
print(f"If current state is s_1 (H, one of the firms exceeded 35), firm 2 should produce: {pi_2_b(1)[0]}")
print(f"If current state is s_2 (L, neither firm exceeded exceeded 35), firm 2 should produce: {pi_2_b(2)[0]}")
print(f"If current state is s_3 (L, one of the firms exceeded 35), firm 2 should produce: {pi_2_b(3)[0]}") 

If current state is s_0 (H, neither firm exceeded 35), firm 2 should produce: 35
If current state is s_1 (H, one of the firms exceeded 35), firm 2 should produce: 34
If current state is s_2 (L, neither firm exceeded exceeded 35), firm 2 should produce: 26
If current state is s_3 (L, one of the firms exceeded 35), firm 2 should produce: 19
